# Hands On Day 1 


### In class lab WAP : USE CRISP-DM approach to analyse and prepare data for classification modelling

#### Problem statement
The data provided is from a Personal Loans Campaign executed by a bank.Customers were targeted with an offer of Personal Loans at 10% interest rate. The target variable is whether the customer responded to the campaign and availed the personal loan.
Using the dataset perform the following:


Using the dataset, perform 
1. Check out for null values in the dataset.
2. Describe the data and check all the unique paramenters in the columns & range of important numerical variables
3. Calculate the % of responders vs non-responders - comment on whether the data is balanced or imbalanced
4. Analyse how the  % of responders vary by - Gender, Occupation
5. Build a demographic profile of the banks's customers describing their age range, gender distribution and occupation
6. Analyse how the following factors vary with the target variable:
   * Age
   * Balance
   * LEN_OF_REL
7. Come up with other variables that you think might impact a response from a customer and analyse how they change with the target variable.
8. What model performance measures will you use to evaluate the model - explain the rationale
9. Perform logistic regression to predict if a customer will respond to the campaign and identify the variables that influence the target variable
10. Interpret the logistic regression model summary using Odds ratio

   

### Data Dictionary

In [ ]:
import pandas as pd
dict = pd.read_csv('PL_XSELL2.csv')
dict

In [ ]:
#Import all the useful libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
%matplotlib inline

In [ ]:
#Read the csv file
df = pd.read_csv('PL_XSELL2.csv')

In [ ]:
#Get the column names
df.columns

In [ ]:
#Fetch the number of rows and columns
print ('DataFrame Shape', df.shape)

In [ ]:
#View the top 3 rows of data
df.head(3)

In [ ]:
#Get info about datatype of each column
df.info()

In [ ]:
#Convert relevant columns to categorical variable
df['TARGET'] = df['TARGET'].astype('category')
df['FLG_HAS_CC'] = df['FLG_HAS_CC'].astype('category')
df['FLG_HAS_ANY_CHGS'] = df['FLG_HAS_ANY_CHGS'].astype('category')  
df['FLG_HAS_NOMINEE'] = df['FLG_HAS_NOMINEE'].astype('category')
df['FLG_HAS_OLD_LOAN'] = df['FLG_HAS_OLD_LOAN'].astype('category')

#Get info about datatype of each column
df.info()

In [ ]:
#Obtain summary of columns with numerical data
df.describe()

### Phase #3 Data Preparation


Performing some data cleaning, validation, and sanity checks before performing any analysis

In [ ]:
df.columns

In [ ]:
#Drop redundant columns
df = df.drop(['CUST_ID'], axis=1) 

#### Missing Data

In [ ]:
### Checking for missing values
df.isnull().sum()

No missing data

In [ ]:
#Sort customers from oldest to newest
df.sort_values(by='LEN_OF_REL', ascending=False).head(5)

#### Outliers and Distributions

To get a good understanding of the questions that are being asked, it may be necessary to remove projects with very small and large project goal's

In [ ]:
figsize = (18,6)

def histogram_plot(dataset, column, x_label, title):
    '''
    Plots histogram of input feature
    
    INPUT
    dataset = dataset with feature that is to be plotted
    column = feature of dataset to be plotted
    x_label = Label title of the x axis
    title = Plot figure title
    
    OUTPUT
    Distribution plot
    '''
    plt.figure(figsize=figsize);
    plt.hist(data = dataset, x = column, bins = 20);
    plt.xlabel(x_label);
    plt.grid(False)
    plt.title(title)

In [ ]:
##distribution of account balance


histogram_plot(df, 'BALANCE', 'Account Balance INR', 'Customer distribution')

It is an extremely left skewed distribtuion

In [ ]:
###Project Length Distribution

figsize = (18,6)


histogram_plot(df, 'LEN_OF_REL', 'Length of Relationship in months', 'Account Length Distribution')

The distribution is uniform

## Analysis - Finding Answers

### Calculate the % of responders vs non-responders - comment on whether the data is balanced or imbalanced

In [ ]:
print ('Unique Categories of Target: ',df.TARGET.nunique())

#### % of positive response = # of projects with TARGET=1 / total rows

In [ ]:
# % of positive & negative responses in the Y variable
df.TARGET.value_counts()/(len(df))*100

### Analyse how the % of responders vary by - Gender

In [ ]:
gender_perc = round(pd.crosstab(df.TARGET, df.GENDER, normalize='columns'),2)
gender_perc

### Response is higher for Men compared to Women

### Analyse how the  % of responders vary by - Occupation

In [ ]:
occ_perc = round(pd.crosstab(df.TARGET, df.OCCUPATION, normalize='columns'),2)
occ_perc

#### Response is highest for Self employed

## Build a demographic profile of the banks's customers describing their age range, gender distribution and occupation


In [ ]:
print(df.AGE.describe())
print(df.OCCUPATION.value_counts())
print(df.GENDER.value_counts())

### Analyse how the following factors vary with the target variable:
   * Age
   * Balance
   * LEN_OF_REL

In [ ]:
### AGE vs TARGET
sns.boxplot(x="AGE", y="TARGET", data=df)

In [ ]:
### BALANCE vs TARGET
sns.boxplot(x="BALANCE", y="TARGET", data=df)

In [ ]:
### LEN OF REL vs TARGET
sns.boxplot(x="LEN_OF_REL", y="TARGET", data=df)

In [ ]:
#SCR vs target
### SCR vs TARGET
sns.boxplot(x="SCR", y="TARGET", data=df)

In [ ]:
sns.boxplot(x="NO_OF_L_CR_TXNS", y="TARGET", data=df)

In [ ]:
sns.boxplot(x="NO_OF_L_DR_TXNS", y="TARGET", data=df)

In [ ]:
#Ability to hold money in the account 
sns.boxplot(x="HOLDING_PERIOD", y="TARGET", data=df)


In [ ]:
acc_perc = round(pd.crosstab(df.TARGET, df.ACC_TYPE, normalize='columns'),2)
acc_perc

In [ ]:
flag_perc = round(pd.crosstab(df.TARGET, df.FLG_HAS_CC, normalize='columns'),2)
flag_perc

In [ ]:
flag1_perc = round(pd.crosstab(df.TARGET, df.FLG_HAS_ANY_CHGS, normalize='columns'),2)
flag1_perc

### Perform logistic regression to predict if a customer will respond to the campaign and identify the variables that influence the target variable

In [ ]:
model_df = df.copy()

In [ ]:
model_df['TARGET'].value_counts()

In [ ]:
model_df.columns

In [ ]:
#Drop variables with near zero variance
threshold = 0.01

model_df = model_df.drop(model_df.std()[model_df.std() < threshold].index.values, axis=1)
print(model_df.shape)

In [ ]:
#Find highly correlated columns
#Create a subset of numerical variables
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

newdf = model_df.select_dtypes(include=numerics)

In [ ]:
#Function to get top absolute correlations
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(newdf, 10))

In [ ]:
#delete certain columns before model building
model_df = model_df.drop(['AMT_L_DR',"AVG_AMT_PER_ATM_TXN"], axis=1)

Preprocessing Data

In [ ]:
#X and Y variables
X  =  model_df.drop(['TARGET'], axis=1)
y  =  model_df[['TARGET']]

print(X.shape)
print(y.shape)

In [ ]:
#Convert categorical vriables to dummy variables
X = pd.get_dummies(X, drop_first=True)

In [ ]:
X.head()

In [ ]:
##Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

In [ ]:
#Logistic regression using stats model
import statsmodels.api as sm

logit = sm.Logit(y_train, sm.add_constant(X_train))
lg = logit.fit()

In [ ]:
#Summary of logistic regression
lg.summary()

In [ ]:
#Calculate Odds Ratio
#Calculate Odds Ratio
lgcoef = pd.DataFrame(lg.params, columns=['coef'])
lgcoef.loc[:, "Odds_ratio"] = np.exp(lgcoef.coef)
lgcoef['pval']=lg.pvalues
pd.options.display.float_format = '{:.2f}'.format
lgcoef
lgcoef.to_csv('logit_handson.csv')

In [ ]:
lgcoef

In [ ]:
#Calculate Odds Ratio
lgcoef = pd.DataFrame(lg.params, columns=['coef'])
lgcoef.loc[:, "Odds_ratio"] = np.exp(lgcoef.coef)
lgcoef['probability'] = lgcoef['Odds_ratio']/(1+lgcoef['Odds_ratio'])
lgcoef['pval']=lg.pvalues
pd.options.display.float_format = '{:.2f}'.format
lgcoef

lgcoef = lgcoef.sort_values(by="Odds_ratio", ascending=False)
pval_filter = lgcoef['pval']<=0.05
lgcoef[pval_filter]
